**Assignment 1**

In [4]:
import string
import chromadb
from sentence_transformers import SentenceTransformer

In [15]:
def preprocess(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

In [16]:
# Hardcoded documents
doc = [
    "Machine learning lets computers learn without being programmed.",
    "Natural language processing helps machines understand human language.",
    "Deep learning uses neural networks to solve problems.",
    "ChromaDB stores and retrieves vector embeddings efficiently.",
    "Google Colab is a free tool for coding and machine learning."
]

In [17]:
clean_docs = [preprocess(docs) for docs in doc]

In [19]:
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(clean_docs).tolist()

In [22]:
client = chromadb.Client()
if "mini_rag" in [col.name for col in client.list_collections()]:
    client.delete_collection("mini_rag")

collection = client.create_collection("mini_rag")

collection.add(documents=clean_docs, embeddings=doc_embeddings, ids=[str(i) for i in range(len(doc))])

In [23]:
query = input("Ask a question: ")
query_vec = model.encode([preprocess(query)]).tolist()

Ask a question: What is Google Colab used for?


In [25]:
result = collection.query(query_embeddings=query_vec, n_results=1)
print("Most relevant document:\n", result['documents'][0][0])

Most relevant document:
 google colab is a free tool for coding and machine learning


**Assignment 2**

In [67]:
!pip install -q rdflib sentence-transformers chromadb

In [68]:
from rdflib import Graph, Namespace, RDF
from sentence_transformers import SentenceTransformer
import chromadb

In [69]:
g = Graph()
EX = Namespace("http://example.org/")

In [70]:
g.add((EX.Sham, RDF.type, EX.Person))
g.add((EX.Ram, RDF.type, EX.Person))
g.add((EX.Sham, EX.knows, EX.Ram))
g.add((EX.Ram, EX.worksAt, EX.ABC))
g.add((EX.Sham, EX.worksAt, EX.XYZLtd))

<Graph identifier=N78c242f476cf4e9ea535902eeffe78ef (<class 'rdflib.graph.Graph'>)>

In [71]:
sentences = [f"{s.split('/')[-1]} {p.split('/')[-1]} {o.split('/')[-1]}" for s, p, o in g]

In [72]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences).tolist()

In [73]:
client = chromadb.Client()

In [74]:
if "kg_collection" in [col.name for col in client.list_collections()]:
    client.delete_collection("kg_collection")
collection = client.create_collection("kg_collection")
collection.add(documents=sentences, embeddings=embeddings, ids=[str(i) for i in range(len(sentences))])

In [77]:
query = input("Ask a question: ")
query_embedding = model.encode([query]).tolist()

Ask a question: Who does Sham know?


In [78]:
result = collection.query(query_embeddings=query_embedding, n_results=1)
print("Most relevant fact:\n", result['documents'][0][0])

Most relevant fact:
 Sham knows Ram
